In [ ]:
##########################################################
#                                                        #
#        Genome-wide CRISPR screen in human T cells      #
#              reveals regulators of FOXP3               #
#                                                        #
##########################################################
#                                                        #
#          Linear Regression at the gene level           #
#                                                        #
##########################################################

In [ ]:
# Libraries

import glob,os
import numpy as np
import scipy.stats as sp_stats
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from statsmodels.distributions.empirical_distribution import ECDF
import argparse
import fastcluster
from scipy.cluster.hierarchy import dendrogram
import seaborn as sns
from statsmodels.stats.multitest import multipletests
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score
from adjustText import adjust_text
import scanpy as sc
import pandas as pd
import re

In [ ]:
# Load functions 

# From https://github.com/klarman-cell-observatory/Perturb-CITE-seq
def fit_lm(X, y, l1_ratio=0.5, alpha=0.0005, max_iter=10000, z_score=False):
    lmfit = ElasticNet(precompute=True, l1_ratio=l1_ratio, alpha=alpha, max_iter=max_iter)
    if z_score:
        y = sp_stats.zscore(y, axis=0)
    lmfit.fit(X, y)
    return lmfit.coef_, lmfit


# From MIMOSCA notebook (https://github.com/asncd/MIMOSCA/blob/master/contrived-em_example.ipynb)
def bayes_cov_col(Y,X,cols,lm):
    """
    @Y    = Expression matrix, cells x x genes, expecting pandas dataframe
    @X    = Covariate matrix, cells x covariates, expecting pandas dataframe
    @cols = The subset of columns that the EM should be performed over, expecting list
    @lm   = linear model object
    """
    #EM iterateit
    Yhat=pd.DataFrame(lm.predict(X))
    Yhat.index=Y.index
    Yhat.columns=Y.columns
    SSE_all=np.square(Y.subtract(Yhat))
    X_adjust=X.copy()
    df_SSE   = []
    df_logit = []
    for curcov in cols:
        curcells=X[X[curcov]>0].index
        if len(curcells)>2:
            X_notcur=X.copy()
            X_notcur[curcov]=[0]*len(X_notcur)
            X_sub=X_notcur.loc[curcells]
            Y_sub=Y.loc[curcells]
            GENE_var=2.0*Y_sub.var(axis=0)
            vargenes=GENE_var[GENE_var>0].index
            Yhat_notcur=pd.DataFrame(lm.predict(X_sub))
            Yhat_notcur.index=Y_sub.index
            Yhat_notcur.columns=Y_sub.columns
            SSE_notcur=np.square(Y_sub.subtract(Yhat_notcur))
            SSE=SSE_all.loc[curcells].subtract(SSE_notcur)
            SSE_sum=SSE.sum(axis=1)
            SSE_transform=SSE.div(GENE_var+0.5)[vargenes].sum(axis=1)
            logitify=np.divide(1.0,1.0+np.exp(SSE_transform))#sum))
            df_SSE.append(SSE_sum)
            df_logit.append(logitify)
            X_adjust[curcov].loc[curcells]=logitify
    return X_adjust

In [ ]:
# Linear Regression at the gene level

    ## gRNA correlation results
pval = pd.read_csv('Pvalues_gRNAs.txt', sep='\t')
pval.sel = pval.copy()
pval.sel = pval.sel[pval.sel['adj.pval'] <= 0.5]

    ## Construct the final covariate matrix
X = pd.read_csv("Xprime_matrix.txt", sep="\t", index_col = 0)

        # Separate in two covariate matrices
names_cov = X.columns
r = re.compile("gRNA")
grna = list(filter(r.match, names_cov))
X_gRNA = pd.DataFrame(X, columns = grna)
others = list(np.setdiff1d(names_cov, grna, assume_unique = False))
X_others = pd.DataFrame(X, columns = others)

        # Select the correct gRNAs
gRNA1 = list(pval.sel['gRNA1'])
gRNA2 = list(pval.sel['gRNA2'])
good_grna = list(set(gRNA1) | set(gRNA2))
X_gRNA = pd.DataFrame(X_gRNA, columns = good_grna)
max_row = {'CellBarcode': X_gRNA.index.values, 'Max': X_gRNA.max(axis=1), 'gRNA': X_gRNA.idxmax(axis=1)}
max_df = pd.DataFrame(max_row)
max_df = max_df[max_df['Max'] > 0]
gene = []
for i in max_df['gRNA']:
    a = i.split('_')[1]
    b = a.count('-')
    if b == 1:
        c = a.split('-')[0]
    elif b > 1:
        c = a.split('-')[0] + '-' + a.split('-')[1]
    gene.append(c)

max_df['Gene'] = gene

df_cov = pd.DataFrame(max_df, index=max_df['CellBarcode'], columns=['Gene'])
dms = pd.get_dummies(df_cov[['Gene']])
print(dms)

X_others = pd.DataFrame(X_others, index=dms.index.values)
Xfinal = pd.concat([X_others, dms], axis=1)

    ## Expression matrix (targets)
expr_mtx = pd.read_csv("Y_matrix.txt", sep="\t", index_col = "CellBarcode")
expr_mtx = pd.DataFrame(expr_mtx, index=dms.index.values)
print(expr_mtx)

In [ ]:
   ## Elasticnet regularization
coeff_mtx, fit_lm_1 = fit_lm(X=Xfinal, y=expr_mtx, alpha=0.0001)
coeff_mtx = np.transpose(coeff_mtx)
coeff_df = pd.DataFrame(coeff_mtx, columns = list(expr_mtx.columns), index = list(Xfinal.columns))
coeff_df.to_csv('Beta_matrix_LR3.txt', sep='\t')

In [ ]:
# Reassign target genes (adjust covariates)
names_cov = Xfinal.columns
r = re.compile("Gene")
newlist = list(filter(r.match, names_cov))
print(newlist)
X_adjust = bayes_cov_col(expr_mtx, Xfinal, newlist, fit_lm_1)
X_adjust_df = pd.DataFrame(X_adjust, columns = list(Xfinal.columns), index = list(Xfinal.index))
print(X_adjust)
X_adjust.to_csv('Xsecond_matrix.txt', sep='\t')

In [ ]:
# Final Linear Regression
coeff_mtx_2, fit_lm_2 = fit_lm(X=X_adjust, y=expr_mtx, alpha=0.0001)
coeff_mtx_2 = np.transpose(coeff_mtx_2)
coeff_df_2 = pd.DataFrame(coeff_mtx_2, columns = list(expr_mtx.columns), index = list(X_adjust.columns))
coeff_df_2.to_csv('Beta_matrix_LR4.txt', sep='\t')